# Classificador de contas contábeis
O objetivo deste projeto é criar um classificador de contas contábeis para ajudar a identificar e categorizar lançamentos financeiros de uma empresa. Nessa primeira etapa, vamos explorar e analisar os dados para entender melhor o problema e identificar as características relevantes para o classificador.

## Quais serão os próximos passos?
- Aumentar a probabilidade de acerto para acima de 70%
- Preencher a conta contábil quando estiver vazia e a probabilidade for maior de 70%. Caso contrário, dê uma sugestão da conta a ser preenchida na coluna "Probabilidade".
- Criar um relatório de classificação com a matriz de confusão
- Preencher a conta contábil e a probabilidade, gerando uma nova planilha com os resultados

## Bibliotecas utilizadas
- Pandas: carregamento, manipulação e tratamento de dados, transformando-os em DataFrame
- Numpy: computação numérica
- Scikit-learn: para o desenvolvimento do projeto (treinamento e avaliação do modelo)
- Nltk: ferramenta para processamento de texto (stopwords)
- Matplotlib e Seaborn: visualização dos dados

In [ ]:
# - Bibliotecas para manipulação e análise de dados
import pandas as pd
import numpy as np
import openpyxl

# - Bibliotecas para visualização de dados
import matplotlib.pyplot as plt
import seaborn as sns

# - Bibliotecas para ML
import nltk
nltk.download('stopwords')
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix

# - Configurações de visualização
%matplotlib inline
sns.set_style("whitegrid")
plt.style.use("fivethirtyeight")

print("Bibliotecas importadas com sucesso!")

## DataFrame com os dados
A planilha a ser utilizada será **teste_contabeis_2024.xlsx**. Ela possui 5 colunas

 Colunas         | Valores                                      |
|--------------------|-------------------------------------------------------|
| CONTA             | Número da conta contábil                        |
| BANCO  | Banco vinculado ao lançamento/conta contábil            |
| DATA            | Data do lançamento           |
| DESCRIÇÃO DO LANÇAMENTO | Descrição do lançamento. Ex: Pagto. Salário  |  
| VALOR | Valor do lançamento|

Antes de iniciarmos as análises e testes de modelo, vamos verificar nosso *dataset*, analisando as 5 primeiras entradas e as dimensões do mesmo.

In [ ]:
# Criando o DataFrame
df = pd.read_excel('teste_contabeis_2024.xlsx')

# Exibindo as primeiras linhas do DataFrame
df.head()

In [ ]:
# Mostrando a dimensão do DataFrame
print(f"O DataFrame possui {df.shape[0]} linhas e {df.shape[1]} colunas.")
print("--------------------------------")
# Informações técnicas do DataFrame
df.info()
print("--------------------------------")
# pegando colunas e colocando em dataset
dataset = df.columns.tolist()
print(dataset)

print("--------------------------------")
# Verificando contas contábeis a serem classificadas
num_classes = df['CONTA'].nunique()
print(f"Número de contas contábeis únicas: {num_classes}")
print("--------------------------------")

## Como as contas estão distribuídas?
Criando um *DataFrame* para mostrar a distribuição de lançamentos por conta contábil. Logo em seguida, plotarei esses dados para melhor visualização

In [ ]:
# Montagem de grafico de barras para visualizar a distribuição das classes
print("Distribuição de lançamentos por conta contábil: ")
count_contas = df['CONTA'].value_counts()
print(count_contas)
print("--------------------------------")
# Plotando o gráfico de barras
plt.figure(figsize=(15, 8))
sns.barplot(x=count_contas.index, y=count_contas.values, palette="viridis")
plt.title("Distribuição de Lançamentos por Conta Contábil")
plt.xlabel("Conta Contábil")
plt.ylabel("Número de Lançamentos")
plt.xticks(rotation=90)
plt.show()


## Como podemos tratar o texto para fazer a classificação?
Para tratar o texto, vamos utilizar o *nltk* para limpar o texto com as *stopwords*.

- **Stopwords**: são as palavras mais comuns e frequentes em um idioma ou contexto, que servem como conexão para formar uma oração ou frase. Elas podem ser removidas do texto para melhorar a qualidade da classificação.

O *dataset* a seguir faz a comparação do histórico original com o histórico limpo.

In [ ]:
# Tratamento de texto - descrição
import re
from nltk.corpus import stopwords

# função para limpar o texto
def clean_text(text):
    # transformar em minúsculas
    text = str(text).lower()
    # remover numeros
    text = re.sub(r'\d+', ' ', text)
    # remover pontuação e caracteres especiais
    text = re.sub(r'[^\w\s]', ' ', text)
    # remover espaços extras
    text = text.strip()
    text = re.sub(r'\s+', ' ', text)
    # remover stopwords
    stopwords_pt = set(stopwords.words('portuguese'))
    # Opcional: adicionar stopwords condizentes ao contexto
    custom_stopwords = {'Lançamento', 'pagto', 'aplicações'}
    # update
    stopwords_pt.update(custom_stopwords)
    # Separar as palavras
    words = text.split()
    # Remover as stopwords
    words_filtered = [word for word in words if word not in stopwords_pt and len(word) > 2]
    # Juntar as palavras novamente
    clean_text = ' '.join(words_filtered)
    return clean_text

print("Limpando os textos das descrições...")
# Aplicando a função de limpeza ao DataFrame
# Criando uma coluna com o texto limpo
df["DESCRIÇÃO_LIMPA"] = df["DESCRIÇÃO DO LANÇAMENTO"].apply(clean_text)
# Comparação
print("Comparação entre texto original e texto limpo:")
df[["DESCRIÇÃO DO LANÇAMENTO", "DESCRIÇÃO_LIMPA"]].head(10)

## Quais são as palavras mais frequentes?
O *dataset* a seguir mostra as palavras mais frequentes após o tratamento do histórico.

In [ ]:
# Análise de frequência das palavras

count_vectorizer = CountVectorizer()
# matriz de contagem de palavras
arr_count = count_vectorizer.fit_transform(df["DESCRIÇÃO_LIMPA"])
# Somando as ocorrências de cada palavra
sum_words = arr_count.sum(axis=0)
# Criando um dicionário de palavras e suas frequências
words_freq = [(word, sum_words[0, idx]) for word, idx in count_vectorizer.vocabulary_.items()]
# Ordenar lista da mais frequente para a menos frequente
words_freq_sorted = sorted(words_freq, key=lambda x: x[1], reverse=True)
# Criando DataFrame
df_words_freq = pd.DataFrame(words_freq_sorted, columns=['Palavra', 'Frequência'])

# Mostrando as palavras mais frequentes
print("Palavras mais frequentes nas descrições limpas:")
df_words_freq.head(50) 

## Criando o modelo
Antes de criar o modelo, é criado um filtro para prepará-lo. A divisão do modelo deve ter estratificação com pelo menos 5 ocorrências por classe. A divisão entre dados é composta por: **70% treino e 30% teste**.
Lembrando que:

##### **X = coluna com histórico**
##### **y = coluna com as contas contábeis**

In [ ]:
# Criando filtro para preparar modelo. A divisão do modelo deve ter estratificação com pelo menos 5 ocorrências por classe
count_acc = df['CONTA'].value_counts()
valid_acc = count_acc[count_acc >= 5].index
df_filtered = df[df['CONTA'].isin(valid_acc)]

# Criando primeiro modelo de teste
X = df_filtered["DESCRIÇÃO_LIMPA"]
y = df_filtered["CONTA"]
# Divisão entre dados de treino e teste: 70% treino e 30% teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
# Definir e criar pipeline
text_clf_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', MultinomialNB()),
])   
# Treinando o pipeline
text_clf_pipeline.fit(X_train, y_train)
print("Modelo treinado com sucesso!")    

# Relatorio de classificação
O relatório nos mostrará o resultado da classificação dividido em 4 colunas:

- **Precision**: de toda a classificação, qual a porcentagem de acertos? (Mede a qualidade da previsão)
- **Recall**: de todos os exemplos que realmente eram de *X*, quantos o modelo conseguiu encontrar? (Mede a quantidade de acertos)
- **F1-Score**: uma média entre *Precision* e *Recall*. Ótima métrica para verificar o balanceamento do *dataset*
- **Accuracy**: Acurácia é a porcentagem de acertos em relação ao total de previsões
(Acurácia = Número de acertos / Número total de amostras)

## Matriz de confusão
Para ter uma visão mais detalhada de onde tivemos acertos e erros, usaremos a Matriz de Confusão

- Linhas representam a conta real
- Colunas representam a conta prevista pelo modelo
- Os números na diagonal principal são os acertos
- Qualquer número fora da diagonal representa um erro de classificação

In [ ]:
# Avaliando o modelo, criando matriz de confusão e relatório de classificação
# Usando o pipeline para fazer previsões
y_pred = text_clf_pipeline.predict(X_test)
# Imprimindo o relatório de classificação
print("Relatório de Classificação: ")
print(classification_report(y_test, y_pred))
# Gerar e visualizar matriz de confusão
conf_matrix = confusion_matrix(y_test, y_pred)
labels = sorted(y_test.unique())
plt.figure(figsize=(12, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels)
plt.ylabel('Conta Real')
plt.xlabel('Conta Prevista')
plt.title('Matriz de Confusão')
plt.show()

## Análise de keywords
Aqui será analisado a distribuição das keywords, dividindo seus resultados em 3 colunas:

- Keyword: são as palavras consideradas como principais na classificação do lançamento
- Frequência total: quantas vezes elas aparecem
- Conta dominante: nas vezes em que aparecem, em quais contas seus lançamentos costumam ser atribuídos
  
Através desses dados, é possível prever quais palavras são mais determinantes e menos ambíguas na hora de realizar a classificação dos lançamentos

In [ ]:
# Analisando a distribuição das keywords, verificando se pertencem a mais de uma conta
from tqdm.auto import tqdm # barra de progresso
print("Iniciando análise de palavras-chave...")
df_analysis = df_filtered.copy()
# Extraindo todo o vocabulário
vectorizer_vocab = CountVectorizer(min_df=2)
vectorizer_vocab.fit(df_analysis["DESCRIÇÃO_LIMPA"])
vocab = vectorizer_vocab.get_feature_names_out()
print(f"Vocabulário extraído com {len(vocab)} palavras.")

# Iterar sobre cada palavra e calcular as estatísticas
analysis_list = []
for word in tqdm(vocab, desc="Analisando palavras"):
    # Filtrar linhas que contêm a palavra
    df_word = df_analysis[df_analysis["DESCRIÇÃO_LIMPA"].str.contains(rf'\b{word}\b', regex=True)]
    if not df_word.empty:
        total_freq = len(df_word)
        dist = df_word['CONTA'].value_counts()
        dominant_acc = dist.index[0]
        dominant_freq = dist.iloc[0]
        exclusivity = (dominant_freq / total_freq) * 100
        
        # Guarda os resultados
        analysis_list.append({
            "Keyword": word,
            "Frequência Total": total_freq,
            "Conta dominante": dominant_acc,
            "Frequência na conta dominante": dominant_freq,
            "Exclusividade (%)": exclusivity
        })
# Criando DataFrame com os resultados
df_keyword_analysis = pd.DataFrame(analysis_list)
# Ordenando por exclusividade
df_keyword_analysis_sorted = df_keyword_analysis.sort_values(by="Exclusividade (%)", ascending=True)

# Mostrando as palavras com maior exclusividade
print("Palavras com maior exclusividade:")
print(df_keyword_analysis_sorted.head(50))
print("--------------------------------")
# Menos exclusivas
print("Palavras com menor exclusividade:")
print(df_keyword_analysis_sorted[df_keyword_analysis_sorted["Frequência Total"] > 10].sort_values(by="Exclusividade (%)", ascending=False).head(30))

## Probabilidade de previsão
Aqui será mostrado a probabilidade de previsão pela ordem das classes do modelo.

In [ ]:
# Probabilidade de previsão e coluna de classificação
print("Calculando probabilidades de previsão")
# Obter a matriz de probabilidades
arr_proba = text_clf_pipeline.predict_proba(X)
# Ordem da classe conforme o modelo
model_classes = text_clf_pipeline.classes_
print(f"Ordem das classes conforme o modelo: {model_classes}")
# Criação do map para encontrar o índice da classe
class_index_map = {cls: i for i, cls in enumerate(model_classes)}
# Para cada linha do dataset, descobrir qual o indice da classe
# Pegando dados de y, que são as contas reais de cada lançamento
index_real_cls = y.map(class_index_map).values
# Calculando a probabilidade da classe real para cada linha
real_proba = arr_proba[np.arange(len(y)), index_real_cls]
# Criando coluna no DataFrame
df_filtered.loc[:, 'CLASSIFICAÇÃO_PROB'] = real_proba * 100

# Mostrando o resultado final
print("\nResultado final...")
df_display = df_filtered.copy()
df_display['CLASSIFICAÇÃO_PROB'] = df_display['CLASSIFICAÇÃO_PROB'].map('{:.2f}%'.format)

# Exibir colunas mais importantes
columns_to_display = ['DESCRIÇÃO DO LANÇAMENTO', 'DESCRIÇÃO_LIMPA', 'CLASSIFICAÇÃO_PROB']
print(df_display[columns_to_display].head(20))

## Testando inserção de dados
Testada uma nova entrada e retornando a probabilidade de previsão. Nota-se que a probabilidade de previsão vai diminuindo conforme o modelo aprende mais das contas.

In [ ]:
# Testando com novo lançamento
new_entries = ["pagto alexandre dias"]
# Limpando os novos lançamentos
new_entries_clean = clean_text(new_entries)
# Fazendo previsões
proba = text_clf_pipeline.predict_proba([new_entries_clean])
# Obtendo classes previstas
classes = text_clf_pipeline.classes_
# DataFrame mostrando resultados. O .T deixa as contas como linhas
df_results = pd.DataFrame(proba, columns=classes).T
df_results.rename(columns={0: 'Probabilidade'}, inplace=True)
# Convertendo para porcentagem
df_results['Probabilidade'] = df_results['Probabilidade'] * 100
# Ordenando da maior para a menor probabilidade
df_results_sorted = df_results.sort_values(by='Probabilidade', ascending=False)
df_results_sorted['Probabilidade'] = df_results_sorted['Probabilidade'].map('{:.2f}%'.format) 
print("Resultados das previsões para novos lançamentos:")
print(df_results_sorted)

# Mostrando a melhor e pior previsão do modelo
print("Analisando a melhor previsão do modelo...")
# Encontrando a melhor previsão
best_acc = df_results_sorted.iloc[0]
best_proba = df_results_sorted.iloc[0,0]
print(f"Melhor previsão: Conta {best_acc.name} com probabilidade de {best_proba}")

In [ ]:
# Salvando modelo treinado
import joblib
model_name = 'modelo_contabil_pipeline.joblib'
joblib.dump(text_clf_pipeline, model_name)
print(f"Modelo salvo em {model_name}")

## Testando o modelo já salvo
Repetindo os testes anteriores, mas agora usando o modelo salvo.

In [ ]:
# Testando modelo salvo
load_model = joblib.load(model_name)
test_entry = ["TESTE. pagto alexandre dias"]
# Usando modelo carregado para fazer previsão
predict = load_model.predict(test_entry)
proba = load_model.predict_proba(test_entry)
# Pega a probabilidade da classe prevista
max_prob = proba.max()

print("\nTestando modelo salvo...")
print(f"O lançamento '{test_entry[0]}' foi classificado na conta '{predict[0]}' com probabilidade de {max_prob*100:.2f}%")